In [1]:
import os
import re
import nltk
import spacy
import random
import neuralcoref
import numpy as np
import pandas as pd
from tqdm import tqdm
from nltk.stem import WordNetLemmatizer 
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

# Preparation

In [2]:
wnl = WordNetLemmatizer()
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))

In [3]:
nlp = spacy.load('en_core_web_sm')
neuralcoref.add_to_pipe(nlp)

# The SMMRY Algorithm

In [4]:
transition_phrases = ['thus', 'for example', 'for instance', 'namely', 'to illustrate',
                      'in other words', 'in particular', 'specifically', 'such as',
                      'on the contrary', 'contrarily', 'notwithstanding', 'but', 'however',
                      'nevertheless', 'in spite of', 'in contrast', 'yet', 'on one hand',
                      'on the other hand', 'rather', 'or', 'nor', 'conversely', 'at the same time',
                      'while this may be true', 'and', 'in addition to', 'furthermore',
                      'moreover', 'besides', 'than', 'too', 'also', 'both-and', 'another',
                      'equally important', 'second', 'etc.', 'again', 'further', 'last',
                      'finally', 'not only-but also', 'as well as', 'in the second place',
                      'next', 'likewise', 'similarly', 'in fact', 'as a result', 'consequently',
                      'in the same way', 'for example', 'for instance', 'however', 'thus',
                      'therefore', 'otherwise', 'after that', 'afterward', 'then', 'next',
                      'last', 'at last', 'at length', 'at first', 'formerly', 'another', 'finally',
                      'meanwhile', 'at the same time', 'afterwards', 'subsequently',
                      'in the meantime', 'eventually', 'concurrently', 'simultaneously', 'although',
                      'at least', 'still', 'even though', 'granted that', 'while it may be true',
                      'in spite of', 'of course', 'similarly', 'likewise', 'in like fashion',
                      'in like manner', 'analogous to', 'above all', 'indeed', 'of course',
                      'certainly', 'surely', 'in fact', 'really', 'in truth', 'again', 'besides',
                      'also', 'furthermore', 'in addition', 'specifically', 'especially',
                      'in particular', 'to explain', 'to list', 'to enumerate', 'in detail',
                      'namely', 'including', 'for example', 'for instance', 'to illustrate',
                      'thus', 'in other words', 'as an illustration', 'in particular', 'so that',
                      'with the result that', 'consequently', 'hence', 'accordingly', 'for this reason',
                      'therefore', 'because', 'due to', 'as a result', 'in other words', 'then',
                      'therefore', 'finally', 'consequently', 'thus', 'in conclusion', 'as a result',
                      'accordingly', 'for this purpose', 'to this end', 'with this in mind',
                      'with this purpose in mind', 'therefore']

In [5]:
def transition_start(first_sent, dialog_turn):
    if dialog_turn == 1:
        for phrase in transition_phrases:
            if first_sent.lower().startswith(phrase):
                return True
        return False
    else:
        return False

In [6]:
def smmry(text, doc, sent_count, dialog_turn):

    # some preprocessing to omit text within brackets and replace u with you. 
    
    # text = re.sub("[\(\[].*?[\)\]]", "", text)
    # text = text.replace(' u ', ' you ')

    formatted_text = re.sub('[^a-zA-Z]', ' ', text)
    formatted_text = re.sub(r'\s+', ' ', formatted_text)

    # doc = nlp(text)

    fdist = {}
    word_arr = nltk.word_tokenize(formatted_text.lower())

    # preparing a frequency dictionary without considering stop words
    
    for word in word_arr:
        if not word in stop_words:
            word = wnl.lemmatize(word)
            if word not in fdist.keys():
                    fdist[word] = 1
            else:
                    fdist[word] += 1

    sent_arr = nltk.sent_tokenize(text)
    sent_score_arr = []
    summary_arr = []

    sent_arr_coref_resolved = nltk.sent_tokenize(doc._.coref_resolved)

    # compute scores for each sentence

    for sent in sent_arr:
        score = 0
        token_arr = nltk.word_tokenize(sent.lower())
        for word in token_arr:
            word = wnl.lemmatize(word)
            if word in fdist.keys():
                score += fdist[word]

        sent_score_arr.append(score/len(token_arr))

    sent_score_arr = np.array(sent_score_arr)

    all_ind_arr = sent_score_arr.argsort()[-len(sent_score_arr):][::-1]

    ind_arr_unsorted = sent_score_arr.argsort()[-sent_count:][::-1]

    ind_arr = np.sort(ind_arr_unsorted) 

    summary = ''
    changed_first = False

    if len(ind_arr) > 0:

        try:

            ind = ind_arr[0]
            first_sent = sent_arr[ind]

            while (first_sent != sent_arr_coref_resolved[ind] or transition_start(first_sent, dialog_turn)):
                changed_first = True
                for index in all_ind_arr:
                    if index < ind:
                        ind = index
                        break
                first_sent = sent_arr[ind]
                if ind == 0:
                    break
            summary = summary + first_sent + ' '     
            
            if (changed_first):
                first_ind = ind
                sent_score_modified = sent_score_arr[first_ind+1:]
                ind_arr_unsorted = sent_score_modified.argsort()[-(sent_count-1):][::-1]
                ind_arr_next = np.sort(ind_arr_unsorted) 
                
                for i in range(0, len(ind_arr_next)):
                    ind = (first_ind+1) + ind_arr_next[i]
                    if i == len(ind_arr_next) - 1:
                        summary = summary + sent_arr[ind]
                    else:
                        summary = summary + sent_arr[ind] + ' '
            
            else:
                for i in range(1, len(ind_arr)):
                    ind = ind_arr[i]
                    if i == len(ind_arr) - 1:
                        summary = summary + sent_arr[ind]
                    else:
                        summary = summary + sent_arr[ind] + ' '

            return summary

        except Exception as e:

            print("EXCEPTION occured")
            return text

    else:
        print(text)
        print(sent_arr)
        print("EXCEPTION occured: length of sentence array is not > 0")
        return text

# Data Cleaning Functions

In [7]:
# aux_verbs = ['be', 'can', 'could', 'dare', 'do', 'have', 'may', 'might', 'must',
#              'need', 'ought', 'shall', 'should', 'will', 'would']
# wh_words = ['what', 'when', 'where', 'which', 'who', 'whom', 'whose', 'why', 'how']
# q_words = aux_verbs + wh_words

In [8]:
def preprocess_raw(text):
    # Check if text is a str
    if not isinstance(text, str):
        return None

    # Replace HTML escape chars
    text = text.replace('&gt;', '>')
    text = text.replace('&lt;', '<')
    text = text.replace('&amp;', '&')
    text = text.replace('#x200B;', ' ')
    text = text.replace('nbsp;', ' ')

    # Remove brackets
    b_pattern = re.compile(r'(\([^\(\)]*\))|(\[[^\[\]]*\])')
    while b_pattern.search(text):
        text = re.sub(r'(\([^\(\)]*\))|(\[[^\[\]]*\])', '', text)

    # Remove redundant spaces (including breaklines)
    text = ' '.join(text.split())

    # Check if text is empty
    if not text:
        return None

    text_lower = text.lower()

    # Check if text is [deleted] or [removed]
    if text_lower == '[deleted]' or text_lower == '[removed]':
        return None

    # Check if text contains URL
    url_pattern = re.compile(r'[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)')
    if url_pattern.search(text_lower):
        return None

    # Check if text contains 'r/<subreddit>' or 'u/<username>'
    r_pattern = re.compile(r'(^| )\/?r\/[^ ]*')
    if r_pattern.search(text_lower):
        return None
    u_pattern = re.compile(r'(^| )\/?u\/[^ ]*')
    if u_pattern.search(text_lower):
        return None

    # Check if text contains 'reddit'
    if 'reddit' in text_lower:
        return None

    # Check the percentage of alphabetical letters
    num_alphas = 0
    for ch in text:
        if ch.isalpha():
            num_alphas += 1
    if num_alphas / len(text) < 0.7:
        return None

    doc = nlp(text)

    # Check the number of tokens
    if len(doc) < 2:
        return None

    return {'text': text, 'doc': doc}

In [9]:
def preprocess_summary(text):
    # Check if text is a str
    if not isinstance(text, str):
        return None

    # Remove redundant spaces (including breaklines)
    text = ' '.join(text.split())

    # Check if text is empty
    if not text:
        return None

    # Check the percentage of alphabetical letters
    num_alphas = 0
    for ch in text:
        if ch.isalpha():
            num_alphas += 1
    if num_alphas / len(text) < 0.7:
        return None

    doc = nlp(text)

    # Check the number of tokens
    if len(doc) < 2:
        return None

    return {'text': text, 'doc': doc}

In [10]:
def extract_root(text, sent):
#     # Check if the last character is a question mark
#     if sent[-1].text == '?':
#         return None

    if sent.root.pos_ == 'VERB':
#         # Check the first token
#         if sent[0].lemma_.lower() in q_words:
#             return None
        return sent.root.lemma_

    return None

In [11]:
def summarize(preprocessed_text, dialog_turn):
    if preprocessed_text is None:
        return None

    text = preprocessed_text['text']
    doc = preprocessed_text['doc']

    summarized = 0
    sents = [sent for sent in doc.sents]
    if len(sents) > 1:
        summarized = 1
        summary = smmry(text, doc, 1, dialog_turn)
        preprocessed_summary = preprocess_summary(summary)
        if preprocessed_summary is None:
            return None
        summarized_text = preprocessed_summary['text']
        summarized_doc = preprocessed_summary['doc']
        summarized_sents = [sent for sent in summarized_doc.sents]
        if len(summarized_sents) != 1:
            return None
    elif len(sents) == 1:
        summarized_text = text
        summarized_doc = doc
        summarized_sents = sents
    else:
        return None

    if dialog_turn > 1:
        return {'text': summarized_text, 'summarized': summarized, 'length': len(summarized_sents[0])}

    root = extract_root(summarized_text, summarized_sents[0])
    if root is not None:
        return {'text': summarized_text, 'summarized': summarized, 'root': root, 'length': len(summarized_sents[0])}
    else:
        return None

# Filter Submissions

In [34]:
date_suffix = '20200101_20201231'

In [35]:
submission_df = pd.read_csv('../data/reddit/raw/casual_conv_submissions_{}.csv'.format(date_suffix))
submission_df.shape

(97279, 10)

In [36]:
submission_df.head()

,id,created_utc,title,selftext,num_comments,subreddit,subreddit_id,score,author_fullname,link_flair_text
0,ko0y56,1609458941,Fireworks is banned for this newyear here and ...,[removed],0,CasualConversation,t5_323oy,1,t2_4l6jcegg,NaN
1,ko0xfy,1609458866,HAPPY 2021 from Italy!!!,"To those that had it rough,\nTo the ones that ...",17,CasualConversation,t5_323oy,1,t2_5quydde8,:chat: Just Chatting
2,ko0ue3,1609458544,Positivity Blog/Community,[removed],2,CasualConversation,t5_323oy,1,t2_2t9bipf6,NaN
3,ko0uby,1609458537,Can i get some last minute advice?,I know mental health is apparently against the...,13,CasualConversation,t5_323oy,1,t2_7qyyey38,:question: Questions
4,ko0tyb,1609458499,New Year,Hello everyone!\nJust wanted to wish everyone ...,1,CasualConversation,t5_323oy,1,t2_43cjc0iu,:chat: Just Chatting


In [37]:
submission_filtered_cols = ['id', 'summarized', 'from', 'text', 'root', 'length']
submission_filtered_dict = {col: [] for col in submission_filtered_cols}

In [38]:
for i in tqdm(range(submission_df.shape[0])):
    submission_id = submission_df.iloc[i]['id']
    title = submission_df.iloc[i]['title']
    preprocessed_title = preprocess_raw(title)
    summarized_title = summarize(preprocessed_title, dialog_turn = 1)
    if summarized_title is not None:
        submission_filtered_dict['id'].append(submission_id)
        submission_filtered_dict['summarized'].append(summarized_title['summarized'])
        submission_filtered_dict['from'].append('title')
        submission_filtered_dict['text'].append(summarized_title['text'])
        submission_filtered_dict['root'].append(summarized_title['root'])
        submission_filtered_dict['length'].append(summarized_title['length'])
    else:
        selftext = submission_df.iloc[i]['selftext']
        preprocessed_selftext = preprocess_raw(selftext)
        summarized_selftext = summarize(preprocessed_selftext, dialog_turn = 1)
        if summarized_selftext is not None:
            submission_filtered_dict['id'].append(submission_id)
            submission_filtered_dict['summarized'].append(summarized_selftext['summarized'])
            submission_filtered_dict['from'].append('selftext')
            submission_filtered_dict['text'].append(summarized_selftext['text'])
            submission_filtered_dict['root'].append(summarized_selftext['root'])
            submission_filtered_dict['length'].append(summarized_selftext['length'])

  8%|████████                                                                                                    | 7296/97279 [03:00<23:53, 62.78it/s]

EXCEPTION occured


 11%|███████████▌                                                                                               | 10515/97279 [04:17<22:04, 65.52it/s]

EXCEPTION occured


 18%|██████████████████▉                                                                                        | 17161/97279 [07:03<34:06, 39.14it/s]

...

 91%|█████████████████████████████████████████████████████████████████████████████████████████████████▎         | 88514/97279 [37:52<04:20, 33.61it/s]

EXCEPTION occured


 99%|██████████████████████████████████████████████████████████████████████████████████████████████████████████▍| 96721/97279 [41:33<00:09, 60.34it/s]

EXCEPTION occured


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 97279/97279 [41:46<00:00, 38.82it/s]


In [39]:
submission_filtered_df = pd.DataFrame(submission_filtered_dict)

In [40]:
submission_filtered_df

,id,summarized,from,text,root,length
0,ko0xfy,0,selftext,"To those that had it rough, To the ones that b...",wish,41
1,ko0sa9,0,title,Assassins Creed rekindled,rekindle,3
2,ko0nsh,0,title,What's everyone drinking tonight?,drink,6
3,ko0mjr,1,selftext,I also hope everyone achieves a writhing you p...,hope,10
4,ko0j1p,0,title,I Stand Against All Domestic Abuse,stand,6
...,...,...,...,...,...,...
60256,eibhr2,1,selftext,Here take a hug.,take,5
60257,eibeib,0,title,If you need an excuse to separate from the cro...,use,29
60258,eibbz3,0,title,You know when your dogs comfortable with you w...,know,17
60259,eib70r,0,title,In the month of December alone i got over 2000...,get,13


In [41]:
submission_filtered_df.to_csv('../data/reddit/filtered_q/casual_conv_submissions_{}.csv'.format(date_suffix), index = False)

## Batch Process

In [42]:
date_suffices = ['20190101_20191231', '20180101_20181231', '20170101_20171231',
                 '20160101_20161231', '20150101_20151231', '20140608_20141231']
for date_suffix in date_suffices:
    submission_df = pd.read_csv('../data/reddit/raw/casual_conv_submissions_{}.csv'.format(date_suffix))
    print(submission_df.shape)
    submission_filtered_cols = ['id', 'summarized', 'from', 'text', 'root', 'length']
    submission_filtered_dict = {col: [] for col in submission_filtered_cols}
    for i in tqdm(range(submission_df.shape[0])):
        submission_id = submission_df.iloc[i]['id']
        title = submission_df.iloc[i]['title']
        preprocessed_title = preprocess_raw(title)
        summarized_title = summarize(preprocessed_title, dialog_turn = 1)
        if summarized_title is not None:
            submission_filtered_dict['id'].append(submission_id)
            submission_filtered_dict['summarized'].append(summarized_title['summarized'])
            submission_filtered_dict['from'].append('title')
            submission_filtered_dict['text'].append(summarized_title['text'])
            submission_filtered_dict['root'].append(summarized_title['root'])
            submission_filtered_dict['length'].append(summarized_title['length'])
        else:
            selftext = submission_df.iloc[i]['selftext']
            preprocessed_selftext = preprocess_raw(selftext)
            summarized_selftext = summarize(preprocessed_selftext, dialog_turn = 1)
            if summarized_selftext is not None:
                submission_filtered_dict['id'].append(submission_id)
                submission_filtered_dict['summarized'].append(summarized_selftext['summarized'])
                submission_filtered_dict['from'].append('selftext')
                submission_filtered_dict['text'].append(summarized_selftext['text'])
                submission_filtered_dict['root'].append(summarized_selftext['root'])
                submission_filtered_dict['length'].append(summarized_selftext['length'])
    submission_filtered_df = pd.DataFrame(submission_filtered_dict)
    print(submission_filtered_df.shape)
    submission_filtered_df.to_csv('../data/reddit/filtered_q/casual_conv_submissions_{}.csv'.format(date_suffix), index = False)

(75381, 10)


 13%|██████████████                                                                                              | 9782/75381 [04:20<29:34, 36.97it/s]

EXCEPTION occured


 27%|████████████████████████████▊                                                                              | 20288/75381 [09:25<30:13, 30.38it/s]

EXCEPTION occured


...

EXCEPTION occured


 73%|██████████████████████████████████████████████████████████████████████████████▎                            | 12409/16949 [05:06<02:16, 33.18it/s]

EXCEPTION occured


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 16949/16949 [06:48<00:00, 41.46it/s]

(10781, 6)


## Check Questions

In [32]:
questions = []
for i in tqdm(range(submission_filtered_df.shape[0])):
    if submission_filtered_df.iloc[i]['text'].endswith('?'):
        questions.append(submission_filtered_df.iloc[i]['text'])

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 8470/8470 [00:00<00:00, 9562.37it/s]


In [33]:
questions[:10]

['How do you deal with not having family or any close relationships?',
 'Why do I intuitively find this surprising?',
 'How will lack of romantic relationship in your teen and young adult years affect you in the long run?',
 'How will lack of romantic relationship in your teen and young adult years affect you in the long run?',
 'What would teenage you think about who you are now?',
 "Let's chat?",
 'Why do Americans think adults living with parents is bad?',
 "Do you guys feel like life's not that much fun anymore?",
 "Why do schools teach things you don't need to know like algebra?",
 "How can I make sure my mom doesn't feel lonely?"]

# Filter Comments

In [29]:
comment_df = pd.read_csv('../data/reddit/raw/casual_conv_comments_{}.csv'.format(date_suffix))
comment_df.shape

(311185, 9)

In [30]:
comment_df.head()

,id,link_id,parent_id,created_utc,body,subreddit,subreddit_id,score,author_fullname
0,hm5iqwj,t3_r2nmvy,t3_r2nmvy,1637933885,"Yeah, they’re fun! Conscious Club is campy in ...",CasualConversation,t5_323oy,1.0,t2_5yhjayls
1,hm5iup9,t3_r2nlu7,t3_r2nlu7,1637933947,"Sounds like fate, but also disgusting. I hope ...",CasualConversation,t5_323oy,62.0,t2_g7sdcvpm
2,hm5iuth,t3_r2nmvy,t1_hm5iqwj,1637933949,I know!,CasualConversation,t5_323oy,1.0,t2_dvd2sbpi
3,hm5iw7v,t3_r2nlu7,t3_r2nlu7,1637933973,That is an amazing story and even bigger coinc...,CasualConversation,t5_323oy,18.0,t2_xeg58lg
4,hm5j61n,t3_r2nlu7,t1_hm5iup9,1637934133,He’s a doctor now so I hope he’s grown out of ...,CasualConversation,t5_323oy,66.0,t2_5t1dsc17


In [31]:
submission_filtered_df = pd.read_csv('../data/reddit/filtered/casual_conv_submissions_{}.csv'.format(date_suffix))
submission_filtered_ids = submission_filtered_df['id'].tolist()
submission_filtered_ids = ['t3_' + x for x in submission_filtered_ids]

In [32]:
comment_df = comment_df[comment_df['parent_id'].isin(submission_filtered_ids)]
comment_df.shape

(85598, 9)

In [33]:
comment_filtered_cols = ['id', 'parent_id', 'summarized', 'text', 'length']
comment_filtered_dict = {col: [] for col in comment_filtered_cols}

In [34]:
for i in tqdm(range(comment_df.shape[0])):
    comment_id = comment_df.iloc[i]['id']
    parent_id = comment_df.iloc[i]['parent_id']
    body = comment_df.iloc[i]['body']
    preprocessed_body = preprocess_raw(body)
    summarized_body = summarize(preprocessed_body, dialog_turn = 2)
    if summarized_body is not None:
        comment_filtered_dict['id'].append(comment_id)
        comment_filtered_dict['parent_id'].append(parent_id)
        comment_filtered_dict['summarized'].append(summarized_body['summarized'])
        comment_filtered_dict['text'].append(summarized_body['text'])
        comment_filtered_dict['length'].append(summarized_body['length'])

  2%|██▌                                                                                                         | 2051/85598 [00:39<19:59, 69.65it/s]

EXCEPTION occured


 10%|██████████▉                                                                                               | 8871/85598 [03:10<1:39:55, 12.80it/s]

EXCEPTION occured


 12%|█████████████                                                                                              | 10498/85598 [04:05<25:35, 48.89it/s]

...

 88%|██████████████████████████████████████████████████████████████████████████████████████████████▌            | 75680/85598 [28:39<05:12, 31.78it/s]

EXCEPTION occured


 96%|███████████████████████████████████████████████████████████████████████████████████████████████████████    | 82410/85598 [31:08<01:09, 46.06it/s]

EXCEPTION occured


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 85598/85598 [32:13<00:00, 44.27it/s]


In [35]:
comment_filtered_df = pd.DataFrame(comment_filtered_dict)

In [36]:
comment_filtered_df

,id,parent_id,summarized,text,length
0,hm5jftd,t3_r2npa3,0,I take one to two hours a day where I listen t...,24
1,hm5kl30,t3_r2npa3,0,Walking in the countryside withan audiobook.,7
2,hm5klmc,t3_r2nz4v,0,I look outside my window and ask myself this a...,13
3,hm5l7rv,t3_r2npa3,1,I completed a course in Transcendental Meditat...,22
4,hm5q97a,t3_r2nz4v,1,Movies and Video games require thousands of pe...,14
...,...,...,...,...,...
53689,gxn9pkb,t3_n8se6e,1,Good luck!,3
53690,gxnbhpi,t3_n8se6e,0,It's really hard to get work right now so well...,14
53691,gxnxjei,t3_n8se6e,1,Glad you found work.,5
53692,gxocvzh,t3_n8sdhl,1,"I've never lived anywhere with fireflies, and ...",18


In [37]:
comment_filtered_df.to_csv('../data/reddit/filtered/casual_conv_comments_{}.csv'.format(date_suffix), index = False)

## Batch Process

In [43]:
date_suffices = ['20210501_20211231', '20210101_20210430', '20200101_20201231',
                 '20190101_20191231', '20180101_20181231', '20170101_20171231',
                 '20160101_20161231']
for date_suffix in date_suffices:
    comment_df = pd.read_csv('../data/reddit/raw/casual_conv_comments_{}.csv'.format(date_suffix))
    print(comment_df.shape)
    submission_filtered_df = pd.read_csv('../data/reddit/filtered_q/casual_conv_submissions_{}.csv'.format(date_suffix))
    submission_filtered_ids = submission_filtered_df['id'].tolist()
    submission_filtered_ids = ['t3_' + x for x in submission_filtered_ids]
    comment_df = comment_df[comment_df['parent_id'].isin(submission_filtered_ids)]
    print(comment_df.shape)
    comment_filtered_cols = ['id', 'parent_id', 'summarized', 'text', 'length']
    comment_filtered_dict = {col: [] for col in comment_filtered_cols}
    for i in tqdm(range(comment_df.shape[0])):
        comment_id = comment_df.iloc[i]['id']
        parent_id = comment_df.iloc[i]['parent_id']
        body = comment_df.iloc[i]['body']
        preprocessed_body = preprocess_raw(body)
        summarized_body = summarize(preprocessed_body, dialog_turn = 2)
        if summarized_body is not None:
            comment_filtered_dict['id'].append(comment_id)
            comment_filtered_dict['parent_id'].append(parent_id)
            comment_filtered_dict['summarized'].append(summarized_body['summarized'])
            comment_filtered_dict['text'].append(summarized_body['text'])
            comment_filtered_dict['length'].append(summarized_body['length'])
    comment_filtered_df = pd.DataFrame(comment_filtered_dict)
    print(comment_filtered_df)
    comment_filtered_df.to_csv('../data/reddit/filtered_q/casual_conv_comments_{}.csv'.format(date_suffix), index = False)

(311185, 9)
(115319, 9)


  2%|██▍                                                                                                        | 2650/115319 [00:52<28:24, 66.12it/s]

EXCEPTION occured


 10%|██████████▎                                                                                             | 11370/115319 [04:10<2:10:01, 13.32it/s]

EXCEPTION occured


...

EXCEPTION occured


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 246046/246046 [1:36:23<00:00, 42.54it/s]


             id  parent_id  summarized  \
0       ds021wm  t3_7n9jvs           0   
1       ds025kc  t3_7n9jvs           1   
2       ds02aay  t3_7n9jvs           1   
3       ds02coq  t3_7n9jvs           0   
4       ds02ksr  t3_7n9jvs           1   
...         ...        ...         ...   
171230  dbv7exv  t3_5ldl3q           1   
171231  dbv7lns  t3_5ld63x           0   
171232  dbvdday  t3_5ldlxh           1   
171233  dbvzszy  t3_5ldlxh           1   
171234  dbwnr63  t3_5ldln5           1   

                                                     text  length  
0                                          I feel ya, bro       5  
1                 I just completed watching Black Mirror.       7  
2                                       Alone in a crowd.       5  
3           Playing DOOM all night long, and alone too :)      10  
4                                          Happy new year       3  
...                                                   ...     ...  
171230             

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 93/93 [00:01<00:00, 48.93it/s]


         id  parent_id  summarized  \
0   dbumpog  t3_5lcee8           1   
1   dbumrm1  t3_5lce7x           1   
2   dbun0y4  t3_5lce7x           1   
3   dbundan  t3_5lce7x           1   
4   dbuo53y  t3_5l3q41           1   
..      ...        ...         ...   
62  dc6qxsl  t3_4ze20k           1   
63  dc9ec7n  t3_50ya8p           0   
64  dcvb5mp  t3_50mj2d           0   
65  de08pei  t3_539irb           1   
66  dehvel6  t3_51qxh2           1   

                                                 text  length  
0                                   Stay safe though.       4  
1                                             HOORAY!       2  
2            Hope that you're having a nice time too.      10  
3                                     Happy new year!       4  
4                         Check out the wiki for info       6  
..                                                ...     ...  
62                    "Peep Show" gets me every time.       9  
63  Around the World in 80 Days

# Finalize the Submissions and Comments

In [38]:
submission_filtered_df = pd.read_csv('../data/reddit/filtered/casual_conv_submissions_{}.csv'.format(date_suffix))

In [39]:
comment_filtered_df = pd.read_csv('../data/reddit/filtered/casual_conv_comments_{}.csv'.format(date_suffix))
comment_filtered_parent_ids = comment_filtered_df['parent_id'].tolist()
comment_filtered_parent_ids = [x[3:] for x in comment_filtered_parent_ids]

In [40]:
submission_filtered_df = submission_filtered_df[submission_filtered_df['id'].isin(comment_filtered_parent_ids)]
submission_filtered_df.shape

(7941, 6)

In [41]:
final_cols = ['sub_id', 'sub_summarized', 'sub_from', 'sub_text', 'sub_root', 'sub_length',
              'com_id', 'com_summarized', 'com_text', 'com_length']
final_dict = {col: [] for col in final_cols}

In [42]:
for i in tqdm(range(submission_filtered_df.shape[0])):
    sub_id = submission_filtered_df.iloc[i]['id']
    sub_summarized = submission_filtered_df.iloc[i]['summarized']
    sub_from = submission_filtered_df.iloc[i]['from']
    sub_text = submission_filtered_df.iloc[i]['text']
    sub_root = submission_filtered_df.iloc[i]['root']
    sub_length = submission_filtered_df.iloc[i]['length']

    comment_filtered_df_sub = comment_filtered_df[comment_filtered_df['parent_id'] == 't3_' + sub_id]
    for j in range(comment_filtered_df_sub.shape[0]):
        final_dict['sub_id'].append(sub_id)
        final_dict['sub_summarized'].append(sub_summarized)
        final_dict['sub_from'].append(sub_from)
        final_dict['sub_text'].append(sub_text)
        final_dict['sub_root'].append(sub_root)
        final_dict['sub_length'].append(sub_length)
        final_dict['com_id'].append(comment_filtered_df_sub.iloc[j]['id'])
        final_dict['com_summarized'].append(comment_filtered_df_sub.iloc[j]['summarized'])
        final_dict['com_text'].append(comment_filtered_df_sub.iloc[j]['text'])
        final_dict['com_length'].append(comment_filtered_df_sub.iloc[j]['length'])

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7941/7941 [00:46<00:00, 170.12it/s]


In [43]:
final_df = pd.DataFrame(final_dict)

In [44]:
final_df

,sub_id,sub_summarized,sub_from,sub_text,sub_root,sub_length,com_id,com_summarized,com_text,com_length
0,r2v7x0,0,title,My boss told me that during the time my hair w...,make,21,hm6trwr,1,Take it as a compliment.,6
1,r2v7x0,0,title,My boss told me that during the time my hair w...,make,21,hm6tw3v,1,However working at a bar where sex-charged men...,14
2,r2v7x0,0,title,My boss told me that during the time my hair w...,make,21,hm6uegk,1,Why do people get bent out of shape about hair...,15
3,r2v7x0,0,title,My boss told me that during the time my hair w...,make,21,hm6xlf7,1,Will appearing more attractive to customers in...,16
4,r2v7x0,0,title,My boss told me that during the time my hair w...,make,21,hm6zt2c,1,are you suggesting that its a problem for peop...,28
...,...,...,...,...,...,...,...,...,...,...
53689,n8sdhl,0,selftext,I never really got to do things like this when...,get,36,gxkseja,1,Nice man,2
53690,n8sdhl,0,selftext,I never really got to do things like this when...,get,36,gxocvzh,1,"I've never lived anywhere with fireflies, and ...",18
53691,n8sbcb,0,title,After the worst 4 years of my life during coll...,move,22,gxk4z84,0,Not a bad place to live if you can solve for COL,12
53692,n8sbcb,0,title,After the worst 4 years of my life during coll...,move,22,gxk5bpg,1,Good luck on your new adventure!,7


In [45]:
final_df.to_csv('../data/reddit/matched/casual_conv_{}.csv'.format(date_suffix), index = False)

## Batch Process

In [44]:
date_suffices = ['20210501_20211231', '20210101_20210430', '20200101_20201231',
                 '20190101_20191231', '20180101_20181231', '20170101_20171231',
                 '20160101_20161231']
for date_suffix in date_suffices:
    submission_filtered_df = pd.read_csv('../data/reddit/filtered_q/casual_conv_submissions_{}.csv'.format(date_suffix))
    comment_filtered_df = pd.read_csv('../data/reddit/filtered_q/casual_conv_comments_{}.csv'.format(date_suffix))
    comment_filtered_parent_ids = comment_filtered_df['parent_id'].tolist()
    comment_filtered_parent_ids = [x[3:] for x in comment_filtered_parent_ids]
    submission_filtered_df = submission_filtered_df[submission_filtered_df['id'].isin(comment_filtered_parent_ids)]
    print(submission_filtered_df.shape)
    final_cols = ['sub_id', 'sub_summarized', 'sub_from', 'sub_text', 'sub_root', 'sub_length',
                  'com_id', 'com_summarized', 'com_text', 'com_length']
    final_dict = {col: [] for col in final_cols}
    for i in tqdm(range(submission_filtered_df.shape[0])):
        sub_id = submission_filtered_df.iloc[i]['id']
        sub_summarized = submission_filtered_df.iloc[i]['summarized']
        sub_from = submission_filtered_df.iloc[i]['from']
        sub_text = submission_filtered_df.iloc[i]['text']
        sub_root = submission_filtered_df.iloc[i]['root']
        sub_length = submission_filtered_df.iloc[i]['length']

        comment_filtered_df_sub = comment_filtered_df[comment_filtered_df['parent_id'] == 't3_' + sub_id]
        for j in range(comment_filtered_df_sub.shape[0]):
            final_dict['sub_id'].append(sub_id)
            final_dict['sub_summarized'].append(sub_summarized)
            final_dict['sub_from'].append(sub_from)
            final_dict['sub_text'].append(sub_text)
            final_dict['sub_root'].append(sub_root)
            final_dict['sub_length'].append(sub_length)
            final_dict['com_id'].append(comment_filtered_df_sub.iloc[j]['id'])
            final_dict['com_summarized'].append(comment_filtered_df_sub.iloc[j]['summarized'])
            final_dict['com_text'].append(comment_filtered_df_sub.iloc[j]['text'])
            final_dict['com_length'].append(comment_filtered_df_sub.iloc[j]['length'])
    final_df = pd.DataFrame(final_dict)
    print(final_df.shape)
    final_df.to_csv('../data/reddit/matched_q/casual_conv_{}.csv'.format(date_suffix), index = False)

(10084, 6)


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 10084/10084 [01:11<00:00, 141.93it/s]


(73077, 10)
(4738, 6)


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4738/4738 [00:20<00:00, 233.32it/s]


(28280, 10)
(35337, 6)


...

(203263, 10)
(27209, 6)


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 27209/27209 [05:48<00:00, 78.10it/s]


(171235, 10)
(46, 6)


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 46/46 [00:00<00:00, 807.78it/s]

(67, 10)


# Sample Some Pairs

In [31]:
N = final_df.shape[0]
indices = np.sort(np.random.choice(N, 1000, replace = False))
final_df_sample = final_df.iloc[indices]
final_df_sample.to_csv('pushshift/final/casual_conv_20200101_20201231_sample.csv', index = False)

# Some Tests

In [13]:
nlp = spacy.load('en_core_web_sm')

In [23]:
doc = nlp("A silly question")
for sent in doc.sents:
    print(sent.root)
    print(sent.root.pos_)

question
NOUN


In [25]:
doc = nlp("I am happy about it.")
for sent in doc.sents:
    print(sent.root)
    print(sent.root.pos_)

am
AUX


In [26]:
doc = nlp("I went to the market today")
for sent in doc.sents:
    print(sent.root)
    print(sent.root.pos_)

went
VERB


In [27]:
doc = nlp("Just worked more than I ever have in my life")
for sent in doc.sents:
    print(sent.root)
    print(sent.root.pos_)

worked
VERB


In [29]:
doc = nlp("Happy New Year!")
for sent in doc.sents:
    print(sent.root)
    print(sent.root.pos_)

Year
PROPN


In [46]:
doc = nlp("Has she done your homework")
for sent in doc.sents:
    print(sent.root)
    print(sent.root.pos_)
    print(sent[0].lemma_)
    for token in sent:
        print(token, token.lemma_, token.pos_)

done
VERB
have
Has have AUX
she -PRON- PRON
done do VERB
your -PRON- DET
homework homework NOUN


In [35]:
doc = nlp("Did you finish your homework")
sents = [sent for sent in doc.sents]
sent = sents[0]
print(sent)
print(sent.root.pos_)
print(sent[0].lemma_)

Did you finish your homework
VERB
do


In [25]:
doc = nlp("I just gave you the paper")
sents = [sent for sent in doc.sents]
sent = sents[0]
print(len(sent), sent.root, sent.root.pos_, sent.root.lemma_)
print(sent[0].lemma_, sent[0].pos_)

6 gave VERB give
-PRON- PRON


In [13]:
r_pattern = re.compile(r'(^| )\/?r\/[^ ]*')
print(r_pattern.search('ashjs/r/haha__ ss'))

None


In [18]:
re.sub(r'(\([^\(\)]*\))|(\[[^\[\]]*\])', '', 'how about [ashs] and and [[ss]')

'how about  and and ['